In [20]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.covariance import EllipticEnvelope
from sklearn.neighbors import NearestNeighbors
import scipy.stats as stats
from sklearn import preprocessing
from sklearn.neural_network import MLPClassifier

import glob
import os
import scipy as sp
import scipy.stats


from sklearn.covariance import EllipticEnvelope

import scipy.stats as stats

from itertools import product



In [43]:
# Input file name and requested comorbidities
filename = "/Users/yingwang/Google Drive/Study2017/GeorgeMasonUniversity/Fall2018/HAP725/Week12/interpretValue/rawdata.csv"
newfilename = "/Users/yingwang/Google Drive/Study2017/GeorgeMasonUniversity/Fall2018/HAP725/Week12/interpretValue/stat_comorbidity.csv"
full_list_request = 1
request_comorbidities = [[1,0,1],[1,1,1]]
comorbidity_columns = ['Previous MI', 'CHF', 'Shock']
value_columns = 'LOS'


if full_list_request:
    number_col = len(comorbidity_columns)
    all_comb = product(range(2), repeat=number_col)
    request_comorbidities = [i for i in all_comb]

df_requested_comorbidities = pd.DataFrame(columns=comorbidity_columns)
for i in range(len(request_comorbidities)):
    df_requested_comorbidities.loc[i] = request_comorbidities[i]
# print("requested comborbidity is")
# print(df_requested_comorbidities)
request_value_list = []
estimated_list = []

for request in request_comorbidities: 
    df = pd.read_csv(open(filename,'r',errors='ignore'),skip_blank_lines=True, error_bad_lines=False,low_memory=False)
    df_known = df[(df.Clinician == 'Smith')]

    distance_away = (df_known[comorbidity_columns] - np.array(request)).pow(2).sum(1).pow(0.5)
    sum_value = 0
    sum_distance = 0
    estimated = 1

    df_known['distance'] = pd.Series(distance_away, index=df_known.index)

    for index, row in df_known.iterrows():
        if row['distance'] != 0:
            sum_value = row[value_columns]*row['Prob']/distance_away[index] + sum_value
            sum_distance = row['Prob']/distance_away[index] + sum_distance
        else:
            sum_value = row[value_columns]
            sum_distance = 1
            estimated = 0
            break
    
    requested_value = sum_value/sum_distance
    request_value_list.append(requested_value)
    estimated_list.append(estimated)

df_requested_comorbidities['LOS_updated'] = pd.Series(request_value_list, index = df_requested_comorbidities.index)
df_requested_comorbidities['Estimated?'] = pd.Series(estimated_list, index = df_requested_comorbidities.index)

df_requested_comorbidities.to_csv(newfilename)

print(df_requested_comorbidities)


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


  Previous MI CHF Shock  LOS_updated  Estimated?
0           0   0     0     4.000000           0
1           0   0     1     3.000000           0
2           0   1     0     5.000000           0
3           0   1     1     3.000000           0
4           1   0     0     4.000000           0
5           1   0     1     5.000000           0
6           1   1     0     4.500000           0
7           1   1     1     4.118241           1
